# Project Title
### Data Engineering Capstone Project

#### Project Summary

In my capstone project I will revisit the server log data of the platform *Sci-Hub* used by John Bohannon for his analysis of Sci-Hub user behaviour published in Science Magazine 2016 (https://www.sciencemag.org/news/2016/04/whos-downloading-pirated-papers-everyone). Sci-Hub is a shadow library website that provides free access to millions of research papers and books, without regard to copyright, by bypassing publishers' paywalls in various ways. (https://en.wikipedia.org/wiki/Sci-Hub).
The dataset Bohannon was provided by the Sci-Hub owner covers logs from September 2015 to February 2016 and has 28 million log entries.

Using the *March 2020 Public Data File from Crossref*, provided by Crossref on Academic Torrents https://academictorrents.com/details/0c6c3fbfdc13f0169b561d29354ea8b188eb9d63 which includes metadata of 112 million research articles, and mapping it to the Sci-Hub data, I will provide even more explorative options and insights into the behaviour and preferences of Sci-Hub users.

In [ ]:
# Do all imports and installs here
import pandas as pd
import numpy as np
import gzip
import json
import glob
import os
import configparser

### Step 1: Scope the Project

After cleaning the data from the two sources, Sci-Hub and Crossref using the `pandas`library, I will stage the data from both sources in S3. Using a data warehouse in the form of a star schema relational database in Redshift I am going to make the data accessible for analyses via SQL or other suitable tools.

### Step 2: Describe and Gather Data, Explore, Clean and Save: Sci-Hub

The Sci-Hub data consists of 6 text files, one for each month, which are tab-separated:

`sep2015.tab, 
oct2015.tab, 
nov2015.tab, 
dec2015.tab, 
jan2016.tab, 
feb2016.tab`

There are 6 columns of data available, each row is a reference to a download of an article (referenced via the DOI) by a certain user which has further location details.
No column names are provided. I am using `dec2015.tab` in the `scihub_data_raw` folder for an exploration with the `pandas` library. 

In [ ]:
path = "scihub_data_raw/dec2015.tab"

df_scihub = pd.read_table(path,names=["timestamp", "doi", "user_id", "user_country", "user_city", "user_location"],encoding="UTF-8")

The actual content of each of the columns suggests the following schema with column names `timestamp, doi, user_id, user_country, user_city, user_location`:

In [ ]:
df_scihub.head()

`timestamp`, `doi`, and `user_id` seem to be available for almost all rows. `user_country`, `user_country` ànd`user_location` are not provided for all downloads.

In [ ]:
df_scihub.isna().sum()

The function `clean_scihub_data` performs the data cleaning as described in the docstring.

In [ ]:
def clean_scihub_data(df_scihub):
    """
    cleans the data of the Sci-Hub files
    requires a dataframe as input
    creates the 'day' column for partitioning
    removes rows with empty data in columns timestamp, doi and user_id
    """
    df_scihub["day"] = pd.to_datetime(df_scihub["timestamp"]).dt.day
    df_scihub["timestamp"] = pd.to_datetime(df_scihub["timestamp"])
    df_scihub = df_scihub[~df_scihub["timestamp"].isnull()]
    df_scihub = df_scihub[~df_scihub["doi"].isnull()]
    df_scihub = df_scihub[~df_scihub["user_id"].isnull()]
    df_scihub["day"] = df_scihub["day"].astype(int)
    return df_scihub

To clean all files in `scihub_data_raw` (samples in the folder: `dec2015.tab` and `jan2016.tab`) and save them in the `scihub_data`folder in parquet format, partitioned by `day`, I am using the following script:

In [ ]:
path = "scihub_data_raw/"

filenames=glob.glob(os.path.join(path, '*.tab'))
for filename in filenames:
    print("loading {}".format(filename))
    
    df = pd.read_table(filename,names=["timestamp", "doi", "user_id", "user_country", "user_city", "user_location"],encoding="UTF-8")
    
    df = clean_scihub_data(df)
    new_filename = filename.replace("_raw","").replace(".tab",".parquet")
    df.to_parquet(new_filename, partition_cols=["day"], engine='pyarrow')
    print("saved {}".format(new_filename))

I uploaded these files manually to S3 bucket `s3://scihub-data/`

### Step 2: Describe and Gather Data, Explore, Clean and Save: Crossref

The crossref data includes metadata for each DOI registered at Crossref. The file format is JSON, there are 37000 single files.
I will look at one file, `0.json.gz` to analyse the data structure locally. In the project folder `crossref_data_raw` I saved 33 sample files.

In [ ]:
path = "crossref_data_raw/0.json.gz"
with gzip.open(path) as file:
    json_object = json.load(file)
    file.close()

the JSON object in each file contains a list of `items`

In [ ]:
json.dumps(json_object)[:300]

and there are 3000 items in  each of the file

In [ ]:
len(json_object["items"])

one item is a nested JSON with a large amount of metadata, but for our purpose we will not need all of this

In [ ]:
json_object["items"][0]

I am going to extract the columns `"doi", "type", "title", "published-print", "prefix", "publisher", "subject"` including some data cleaning as described in the docstring

In [ ]:
def clean_crossref_data(json_object):
    """
    extracts the items of the Crossref file
    flattens the JSON
    extracts columns:
    "doi", "type", "title", "published-print", "prefix", "publisher", "subject"
    remove all rows where "doi" is null
    """
    df = pd.DataFrame(json_object["items"])
    df = pd.json_normalize(json_object["items"])
    df.columns = df.columns.str.lower()
    df["title"]=df["title"].str[0]
    df["published-print"]=df["published-print.date-parts"].astype(str).str.extract(r'(\d{4})')
    if "subject" in df.columns:
        df["subject"]=df["subject"].str[0]
    else:
        df["subject"]=np.nan
    df=df[~df["doi"].isnull()]

    return df[["doi", "type", "title", "published-print", "prefix", "publisher", "subject"]]

In [ ]:
#call function
df_crossref=clean_crossref_data(json_object)
df_crossref

When looking for Null values, there are some titles and published-print values missing, and unfortunately most of the subjects

In [ ]:
df_crossref.isna().sum()

to clean all files in `crossref_data_raw` and save them in folder `crossref_data` in zipped JSON format, I am using the following snippet:

In [ ]:
path = 'crossref_data_raw/'

filenames=glob.glob(os.path.join(path, '*.gz'))
for filename in filenames:
    print("loading {}".format(filename))
    
    with gzip.open(filename) as file:
        json_object = json.load(file)
        file.close()
        
    df_crossref=clean_crossref_data(json_object)
    
    new_filename = filename.replace("_raw","").replace(".json.gz","_processed.json")
    
    df_crossref.to_json(new_filename, orient='records', lines=True)
    #df_crossref.to_parquet(new_filename, engine='pyarrow')
    
    print("saved {}".format(new_filename))

I uploaded these files manually to S3 bucket `s3://crossref-sample/`

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

First of all, there are two staging tables based on the two sources, Sci-Hub (table name `scihub_data`) and Crossref (table name `crossref_data`)

I decided using a star schema with `downloads` as fact table with supporting dimension tables `time` and `users` which have the SciHub data as source.

`downloads.timestamp` links to `time.timestamp`
`downloads.user_id` links to `users.user_id`


Table `articles` and `publishers` are dimension tables which have Crossref as data source.

`downloads.doi` links to `articles.doi`

As a normalization step I separated the publishers data from the articles to reduce redundancy.

`articles.prefix` links to `publishers.prefix`


Using this schema, you'll be able do explore all aspects of the downloads in a convenient way.

![schema](Capstone_project_schema.png "Schema")


#### 3.2 Mapping Out Data Pipelines

To pipeline the data into the the data model, the following steps are performed:
    
Read the data from the two S3 Buckets and stage it in the Redshift tables `scihub_data` and `crossref_data`

Create the tables and insert the data into Redshift

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Let's assume we have a Redshift cluster with a database created with the neccessary authorizations to connect to S3 stored in `redshift.cfg`

In [ ]:
config = configparser.ConfigParser()
config.read_file(open('redshift.cfg'))

REGION_NAME            = config.get("AWS","REGION_NAME")

#Redshift credentials
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")
DWH_ENDPOINT           = config.get("DWH","DWH_ENDPOINT")
DWH_IAM_ROLE_NAME      = config.get("DWH","DWH_IAM_ROLE_NAME")

##### connect to Redshift database

In [ ]:
%load_ext sql

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT, DWH_DB)
print(conn_string)
%sql $conn_string

##### staging table scihub_data

In [ ]:
%%sql
DROP TABLE IF EXISTS scihub_data;

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS scihub_data (
    timestamp TIMESTAMP NOT NULL, 
    doi varchar(256), 
    user_id varchar(256),
    user_country varchar(256),
    user_city varchar(256),
    user_location varchar(256),
    day INT8); 

In [ ]:
scihub_data_copy="""
COPY scihub_data
FROM 's3://scihub-data/'
CREDENTIALS 'aws_iam_role={}'
FORMAT AS PARQUET;
""".format(DWH_IAM_ROLE_NAME)

In [ ]:
%sql $scihub_data_copy

##### staging table crossref_data

In [ ]:
%sql DROP TABLE IF EXISTS crossref_data;

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS crossref_data (
    doi varchar(256), 
    "type" varchar(256), 
    title varchar(512),
    "published-print" INT,
    prefix varchar(256),
    publisher varchar(256),
    subject varchar(256)); 

In [ ]:
crossref_data_copy="""
COPY crossref_data
FROM 's3://crossref-sample/'
CREDENTIALS 'aws_iam_role={}'
REGION '{}'
JSON AS 'auto';
""".format(DWH_IAM_ROLE_NAME, REGION_NAME)

In [ ]:
%sql $crossref_data_copy

##### downloads table (fact table)

In [ ]:
%sql DROP TABLE IF EXISTS downloads

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS downloads
    (
    download_id INT IDENTITY (0,1),
    timestamp TIMESTAMP NOT NULL,
    doi VARCHAR(256) NOT NULL,
    user_id VARCHAR(256) NOT NULL,
    PRIMARY KEY(download_id)
    );


In [ ]:
%%sql

INSERT INTO downloads 
    ("timestamp",
    "doi",
    "user_id")
SELECT "timestamp",
       "doi",
       "user_id"
FROM scihub_data
WHERE timestamp IS NOT NULL
AND doi IS NOT NULL
AND user_id IS NOT NULL;

##### users table (dimension table)

In [ ]:
%sql DROP TABLE IF EXISTS users;

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS users
    (
    user_id VARCHAR(256) NOT NULL,
    user_country VARCHAR(256),
    user_city VARCHAR(256),
    user_location VARCHAR(256)
    );

In [ ]:
%%sql

INSERT INTO users 
    ("user_id",
    "user_country",
    "user_city",
    "user_location")
SELECT DISTINCT "user_id",
       "user_country",
       "user_city",
       "user_location"
FROM scihub_data
WHERE user_id IS NOT NULL;

##### time table (dimension table)

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS time
    (
    timestamp TIMESTAMP,
    year INT,
    month INT,
    week_of_year INT,
    weekday INT,
    day INT,
    hour INT,
    PRIMARY KEY(timestamp)
    );


In [ ]:
%%sql

INSERT INTO time 
    ("timestamp",
    "year",
    "month",
    "week_of_year",
    "weekday",
    "day",
    "hour")
SELECT DISTINCT timestamp           AS timestamp,
    EXTRACT(year FROM timestamp)    AS year,
    EXTRACT(month FROM timestamp)   AS month,
    EXTRACT(week FROM timestamp)    AS week_of_year,
    EXTRACT(weekday FROM timestamp) AS weekday,
    EXTRACT(day FROM timestamp)     AS day,
    EXTRACT(hour FROM timestamp)    AS hour
FROM scihub_data
WHERE timestamp IS NOT NULL;

##### articles table (dimension table)

In [ ]:
%sql DROP TABLE IF EXISTS articles

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS articles
    (
    doi VARCHAR(256) NOT NULL,
    title VARCHAR(512),
    "published-print" INT,
    prefix VARCHAR(256),
    PRIMARY KEY(doi)
    );


In [ ]:
%%sql

INSERT INTO articles 
    ("doi",
    "title",
    "published-print",
    "prefix")
SELECT DISTINCT "doi",
       "title",
       "published-print",
       "prefix"
FROM crossref_data
WHERE doi IS NOT NULL;

##### publishers table (dimension table)

In [ ]:
%sql DROP TABLE IF EXISTS publishers

In [ ]:
%%sql

CREATE TABLE IF NOT EXISTS publishers
    (
    prefix VARCHAR(256) NOT NULL,
    publisher VARCHAR(256),
    PRIMARY KEY(prefix)
    );

In [ ]:
%%sql

INSERT INTO publishers 
    ("prefix",
    "publisher")
SELECT DISTINCT "prefix",
       "publisher"
FROM crossref_data
WHERE prefix IS NOT NULL;

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

##### data quality check 1: count number of rows

In [ ]:
%sql SELECT count(*) FROM scihub_data;

In [ ]:
%sql SELECT count(*) FROM crossref_data;

In [ ]:
%sql SELECT count(*) FROM downloads;

In [ ]:
%sql SELECT count(*) FROM users;

In [ ]:
%sql SELECT count(*) FROM time;

In [ ]:
%sql SELECT count(*) FROM articles;

In [ ]:
%sql SELECT count(*) FROM publishers;

##### quality check 2: check if primary key is not null

In [ ]:
%sql SELECT count(*) FROM scihub_data WHERE timestamp IS NULL;

In [ ]:
%sql SELECT count(*) FROM crossref_data WHERE doi IS NULL;

In [ ]:
%sql SELECT count(*) FROM downloads WHERE download_id IS NULL;

In [ ]:
%sql SELECT count(*) FROM users WHERE user_id IS NULL;

In [ ]:
%sql SELECT count(*) FROM time WHERE timestamp IS NULL;

In [ ]:
%sql SELECT count(*) FROM articles WHERE doi IS NULL;

In [ ]:
%sql SELECT count(*) FROM publishers WHERE prefix IS NULL;

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

In [ ]:
data_dictionary=pd.read_table("data_dictionary.txt")
data_dictionary

#### Step 5: Complete Project Write Up

**Clearly state the rationale for the choice of tools and technologies for the project.**

Since they datasetes are very large, I choose to store the datasets for the staging tables in S3 because it can be picked up easily by Redshift. Redshift then allows all kinds of querying the data easily using SQL or an UI. Once the data is loaded you can perform analytical queries like:

In [ ]:
#get the number of articles which are found in both dataset

In [ ]:
%%sql 

SELECT count(*)  
FROM downloads d
JOIN articles a
ON d.doi = a.doi;

In [ ]:
#show the most popular titles

In [ ]:
%%sql 

SELECT count(a.title) AS title_count,
       a.title
FROM downloads d
JOIN articles a
ON d.doi = a.doi
GROUP BY a.title
ORDER BY title_count DESC
LIMIT 5;

In [ ]:
# the cities where most users are coming from

In [ ]:
%%sql

SELECT count (user_city) AS user_city_count,
user_city
FROM users
GROUP BY user_city
ORDER BY user_city_count DESC
LIMIT 10;

**Propose how often the data should be updated and why.**

Supposedly we would receive regular and up to date chunks of the server logs from Sci-Hub, e.g. monthly, we would also have to consequently update the Crossref dataset with the most recent articles to have the chance to match as many DOI as possible. There is an Crossref API that could be called, or we could use the latest Public Data File from Crossref available at `Academic Torrents` (currently, the most recent dataset was published in Jan 2021 https://academictorrents.com/details/e4287cb7619999709f6e9db5c359dda17e93d515)

Write a description of how you would approach the problem differently under the following scenarios:
* **The data was increased by 100x.**
    I would increase the size of the Redshift cluster to handle this data
    
* **The data populates a dashboard that must be updated on a daily basis by 7am every day.**
    I would create a pipeline with Airflow that also includes data loading and the cleaning steps I performed, as well as any tasks to update the Redshift tables.
    
* **The database needed to be accessed by 100+ people.**
Since I am using Redshift, I would just need to increase the size of the Redshift cluster